In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from urllib.request import Request, urlopen
import numpy as np
import time
import datetime
import time

2023-04-04 19:06:25.814929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
req = Request('http://192.168.50.145:9000/datasets/btcusdts_300000.csv')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

In [3]:
# Read in the data from the CSV file
coin_train = pd.read_csv(content)

# Split the data into input (X) and output (y) variables
X = coin_train['timestamp']
Y = coin_train[['A', 'a', 'B', 'b']]

In [4]:
# Build a neural network model using Keras
input_layer = Input(shape=1)
x = Dense(32, activation="relu")(input_layer)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
output_layer = Dense(4, activation="linear")(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [5]:
# Train the model on the data
model.fit(X, Y, epochs=10, batch_size=64)

Epoch 1/10
4688/4688 [==============================] - 4s 794us/step - loss: 358805.4375 - accuracy: 0.0214
Epoch 2/10
4688/4688 [==============================] - 4s 789us/step - loss: 363301.0625 - accuracy: 0.0395
Epoch 3/10
4688/4688 [==============================] - 4s 773us/step - loss: nan - accuracy: 0.0250
Epoch 4/10
4688/4688 [==============================] - 4s 774us/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/10
4688/4688 [==============================] - 4s 845us/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/10
4688/4688 [==============================] - 4s 809us/step - loss: nan - accuracy: 0.0000e+00
Epoch 7/10
4688/4688 [==============================] - 4s 836us/step - loss: nan - accuracy: 0.0000e+00
Epoch 8/10
4688/4688 [==============================] - 4s 794us/step - loss: nan - accuracy: 0.0000e+00
Epoch 9/10
4688/4688 [==============================] - 4s 776us/step - loss: nan - accuracy: 0.0000e+00
Epoch 10/10
4688/4688 [============================

In [6]:
model.evaluate(X, Y, verbose=2)

9375/9375 - 5s - loss: nan - accuracy: 0.0000e+00 - 5s/epoch - 488us/step


[nan, 0.0]

In [7]:
X_test = np.array([round(time.time() * 1000)]).reshape(1, 1)
y_pred = model.predict(X_test)

print(y_pred)

1/1 [==============================] - 0s 61ms/step
[[nan nan nan nan]]
